In [2]:
from random import random
import numpy as np
from itertools import combinations
import math
import random

In [3]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [4]:
# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# k: clique size
def generate_graph(num_vertices, pr, k):
    def plant_clique():
        for i in range(k):
            for j in range(i + 1, k):
                for l in range(j + 1, k):
                    a = clique_vertices[i]
                    # print(a)
                    vec[a] += 1
                    b = clique_vertices[j]
                    # print(b)
                    vec[b] += 1
                    c = clique_vertices[l]
                    # print(c)
                    vec[c] += 1
                    add_all(a, b, c, g)

    g = np.array([np.array([np.array([0 for i in range(0, num_vertices)]) for i in range(num_vertices)]) for i in
                  range(num_vertices)])
    vec = np.array([0 for i in range(0, num_vertices)])

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    # clique vertices
    clique_vertices = random.sample(range(num_vertices), k)

    # plant a clique
    plant_clique()
    return g, vec, clique_vertices


In [25]:
a = np.random.uniform(0,1, 1)
a

array([0.56319584])

In [6]:
# n = 4
# p = 0.5
# # res = generate_graph(n, p)

In [7]:
# G = res[0]
# G

In [8]:
# vector storing the flattened list of vertices
# V = res[1]
# V

In [9]:
# np.flip(np.argsort(V))  # sort the indices by the number of presence in hyperedgess in descending order

In [10]:
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [11]:
# remove edge
def remove_edges(g, n_dim, vec, curr):
    vec[curr] = 0
    graph_copy = g.copy()
    for j in range(0, n_dim):
        for k in range(0, n_dim):
            if graph_copy[curr, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr, j, k, graph_copy)
    return graph_copy, vec

In [12]:
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [13]:

def is_clique(g, vec, num_vertices):

    active_count = np.count_nonzero(vec)
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == 6 * math.comb(active_count, 3):
        return True
    return False

In [14]:
n = 10
p = 0.5
k = 5

res = generate_graph(n, p, k)
G = res[0]
G

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 0, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 0, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 0, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0]],

       [[0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
        [1, 0, 1, 0, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 0, 1, 1, 1, 0, 1, 1, 1, 0],
        [1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
        [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
        [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 1, 1, 0]],

       [[0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
        [1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
        [1, 1, 0, 1, 0, 1, 1, 1, 1, 

In [15]:
# vector storing the flattened list of vertices
V = res[1]
V

array([43, 47, 42, 47, 43, 43, 40, 46, 48, 15])

In [16]:
# np.flip(np.argsort(V))

In [17]:
# remove_edges(G, n, V, V[0])

In [18]:
G = res[0]
G

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 0, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 0, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 0, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0]],

       [[0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
        [1, 0, 1, 0, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 0, 1, 1, 1, 0, 1, 1, 1, 0],
        [1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
        [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
        [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 1, 1, 0]],

       [[0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
        [1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
        [1, 1, 0, 1, 0, 1, 1, 1, 1, 

In [19]:
clique_vertices = res[-1]
clique_vertices

[8, 5, 4, 3, 1, 0, 7, 2, 6]

In [21]:
clique = False
while not clique:
    print(V)
    # curr = np.flip(np.argsort(V))[0]
    # V_transformed = np.array([V[i] = math.comb(num_vertices, 2) + 1 if V[i] == 0 for i in range(num_vertices)])

    curr = -1
    
    idx_sorted = np.argsort(V)
    print(idx_sorted)
    for idx in range(n):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            break
    print(curr)
    A = remove_edges(G, n, V, curr)
    print(A[1])
    G = A[0]

    clique = is_clique(G, V, n)

[43 47 42 47 43 43 40 46 48 15]
[9 6 2 0 4 5 7 1 3 8]
9
[41 44 40 43 41 38 38 41 43  0]
[41 44 40 43 41 38 38 41 43  0]
[9 5 6 2 0 4 7 3 8 1]
5
[34 37 33 36 34  0 31 34 36  0]
[34 37 33 36 34  0 31 34 36  0]
[5 9 6 2 0 4 7 3 8 1]
6
[28 31 27 30 28  0  0 28 30  0]
[28 31 27 30 28  0  0 28 30  0]
[5 6 9 2 0 4 7 3 8 1]
2
[23 26  0 25 23  0  0 23 25  0]
[23 26  0 25 23  0  0 23 25  0]
[2 5 6 9 0 4 7 3 8 1]
0
[ 0 22  0 21 19  0  0 19 21  0]
[ 0 22  0 21 19  0  0 19 21  0]
[0 2 5 6 9 4 7 3 8 1]
4
[ 0 19  0 18  0  0  0 16 18  0]
[ 0 19  0 18  0  0  0 16 18  0]
[0 2 4 5 6 9 7 3 8 1]
7
[ 0 17  0 16  0  0  0  0 16  0]
[ 0 17  0 16  0  0  0  0 16  0]
[0 2 4 5 6 7 9 3 8 1]
3
[ 0 16  0  0  0  0  0  0 15  0]


In [22]:
clique_v = res[-1]
clique_v

[8, 5, 4, 3, 1, 0, 7, 2, 6]

In [23]:
V

array([ 0, 16,  0,  0,  0,  0,  0,  0, 15,  0])

## Inclusion Phase